<a href="https://colab.research.google.com/github/EugeneLogvinovsky/ITHillel/blob/main/CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import torch
import math
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10, CIFAR100
from torchvision.transforms import ToTensor

In [74]:
train_ds = CIFAR10("-", train=True, transform=ToTensor(), download=True)
val_ds = CIFAR10("-", train=False, transform=ToTensor(), download=True)

train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=128)

Files already downloaded and verified
Files already downloaded and verified


In [48]:
class Linear(nn.Module):
  def __init__(self, in_features, out_features):
    super(Linear, self).__init__()

    self.in_features = in_features
    self.weight = nn.Parameter(torch.Tensor(in_features, out_features))
    self.bias = nn.Parameter(torch.Tensor(out_features))

    self.reset_parametres()

  def reset_parametres(self):
    std = math.sqrt(2. / self.in_features)
    nn.init.normal_(self.weight, 0, std)
    nn.init.zeros_(self.bias)

  def forward(self, x:torch.Tensor) -> torch.Tensor:
    return x @ self.weight + self.bias


class FFN(nn.Module):
  def __init__(self):
    super(FFN, self).__init__()

    self.net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32*32*3, 512),
    nn.ReLU(),
    
    nn.Linear(512,512),
    nn.ReLU(),

    nn.Linear(512,256),
    nn.ReLU(),

    nn.Linear(256,128),
    nn.ReLU(),

    nn.Linear(128,128),
    nn.ReLU(),   

    nn.Linear(128, 10)


    )

  def forward(self, x:torch.Tensor) -> torch.Tensor:
    return self.net(x)

In [66]:
model = FFN()
opt = optim.SGD(model.parameters(), lr=1e-1)
loss_fn = nn.CrossEntropyLoss()
model.train()
for epoch_int in range(10):
  train_loss, train_acc = [], []
  for (X, Y) in train_dl:
    opt.zero_grad()
    Yp = model(X)
    loss = loss_fn(Yp, Y)
    train_loss.append(loss.detach())
    loss.backward()
    opt.step()

    accuracy = (Yp.argmax(1) == Y).float().mean()
    train_acc.append(accuracy.detach())

  print(f"Epoch{epoch_int}\t TRAIN loss={torch.stack(train_loss).mean()} \t acc={torch.stack(train_acc).mean()}")


model.eval()
for epoch_int in range(10):
  val_loss, val_acc = [], []
  for (X, Y) in train_dl:
    opt.zero_grad()
    Yp = model(X)
    loss = loss_fn(Yp, Y)
    val_loss.append(loss)
    loss.backward()
    opt.step()

    accuracy = (Yp.argmax(1) == Y).float().mean()
    val_acc.append(accuracy)    
  print(f"Epoch{epoch_int}\t EVAL loss={torch.stack(val_loss).mean()} \t acc={torch.stack(val_acc).mean()}")

Epoch0	 TRAIN loss=2.219827651977539 	 acc=0.1592750996351242
Epoch1	 TRAIN loss=2.030822277069092 	 acc=0.23459479212760925
Epoch2	 TRAIN loss=1.8999775648117065 	 acc=0.3057105243206024
Epoch3	 TRAIN loss=1.8127264976501465 	 acc=0.34137627482414246
Epoch4	 TRAIN loss=1.7494510412216187 	 acc=0.364098459482193
Epoch5	 TRAIN loss=1.6871492862701416 	 acc=0.38947010040283203
Epoch6	 TRAIN loss=1.6394222974777222 	 acc=0.4069453477859497
Epoch7	 TRAIN loss=1.5960769653320312 	 acc=0.42792919278144836
Epoch8	 TRAIN loss=1.5554068088531494 	 acc=0.44206759333610535
Epoch9	 TRAIN loss=1.5229648351669312 	 acc=0.4519261419773102
Epoch0	 EVAL loss=1.4864853620529175 	 acc=0.4638267159461975
Epoch1	 EVAL loss=1.4593309164047241 	 acc=0.47783327102661133
Epoch2	 EVAL loss=1.4297394752502441 	 acc=0.4882872700691223
Epoch3	 EVAL loss=1.3975046873092651 	 acc=0.49956440925598145
Epoch4	 EVAL loss=1.3714148998260498 	 acc=0.5080442428588867
Epoch5	 EVAL loss=1.3442362546920776 	 acc=0.51869004964